In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# データの読み込み
modelling_table = pd.read_csv("preprocessed_data.csv")

# valid_datetimeをdatetime型に変換
modelling_table['valid_datetime'] = pd.to_datetime(modelling_table['valid_datetime'])

# 季節ごとのデータフレームを作成
modelling_table['month'] = modelling_table['valid_datetime'].dt.month
spring = modelling_table[modelling_table['month'].isin([3, 4, 5])]
summer = modelling_table[modelling_table['month'].isin([6, 7, 8])]
autumn = modelling_table[modelling_table['month'].isin([9, 10, 11])]
winter = modelling_table[modelling_table['month'].isin([12, 1, 2])]

# 季節ごとの太陽光発電モデル
solar_models = {
    'spring': smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=spring),
    'summer': smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=summer),
    'autumn': smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=autumn),
    'winter': smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=winter)
}

# 季節ごとの風力発電モデル
wind_models = {
    'spring': smf.quantreg('Wind_MWh_credit ~ bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=spring),
    'summer': smf.quantreg('Wind_MWh_credit ~ bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=summer),
    'autumn': smf.quantreg('Wind_MWh_credit ~ bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=autumn),
    'winter': smf.quantreg('Wind_MWh_credit ~ bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=winter)
}

forecast_models_solar = dict()
forecast_models_wind = dict()

# 季節ごとの太陽光発電と風力発電の分位数を計算
for quantile in range(10, 100, 10):
    for season, season_months in {'spring': [3, 4, 5], 'summer': [6, 7, 8], 'autumn': [9, 10, 11], 'winter': [12, 1, 2]}.items():
        forecast_models_solar[f"{season}_q{quantile}"] = solar_models[season].fit(q=quantile/100, max_iter=2500)
        forecast_models_wind[f"{season}_q{quantile}"] = wind_models[season].fit(q=quantile/100, max_iter=2500)
        
        modelling_table.loc[modelling_table['month'].isin(season_months), f"{season}_q{quantile}_solar"] = forecast_models_solar[f"{season}_q{quantile}"].predict(modelling_table.loc[modelling_table['month'].isin(season_months)])
        modelling_table.loc[modelling_table['month'].isin(season_months), f"{season}_q{quantile}_wind"] = forecast_models_wind[f"{season}_q{quantile}"].predict(modelling_table.loc[modelling_table['month'].isin(season_months)])

        # 発電量は0以上、小数点3桁で丸める
        modelling_table.loc[modelling_table[f"{season}_q{quantile}_solar"] < 0, f"{season}_q{quantile}_solar"] = 0
        modelling_table[f"{season}_q{quantile}_solar"] = modelling_table[f"{season}_q{quantile}_solar"].round(2)

        modelling_table.loc[modelling_table[f"{season}_q{quantile}_wind"] < 0, f"{season}_q{quantile}_wind"] = 0
        modelling_table[f"{season}_q{quantile}_wind"] = modelling_table[f"{season}_q{quantile}_wind"].round(2)

# 太陽光と風力の発電量の合計をトータル発電量として計算
for quantile in range(10, 100, 10):
    for season in solar_models:
        modelling_table[f"{season}_q{quantile}"] = modelling_table[f"{season}_q{quantile}_solar"] + modelling_table[f"{season}_q{quantile}_wind"]

# 更新データを保存
modelling_table.to_csv("analyzed_data_sep_sea_rev.csv", index=False)

/var/folders/39/jc66nmzj1ds6kh72kmdgnzn40000gn/T/ipykernel_3063/1260191710.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  modelling_table[f"{season}_q{quantile}"] = modelling_table[f"{season}_q{quantile}_solar"] + modelling_table[f"{season}_q{quantile}_wind"]
/var/folders/39/jc66nmzj1ds6kh72kmdgnzn40000gn/T/ipykernel_3063/1260191710.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  modelling_table[f"{season}_q{quantile}"] = modelling_table[f"{season}_q{quantile}_solar"] + modelling_table[f"{season}_q{quantile}_wind"]
/v

In [2]:
import pandas as pd
import numpy as np

# データの読み込み
modelling_table = pd.read_csv("analyzed_data_sep_sea_rev.csv")

# datetime型に変換
modelling_table['valid_datetime'] = pd.to_datetime(modelling_table['valid_datetime'])
modelling_table['ref_datetime'] = pd.to_datetime(modelling_table['ref_datetime'])

# ピンボール関数の定義
def pinball(y, q, alpha):
    return round((y - q) * alpha * (y >= q) + (q - y) * (1 - alpha) * (y < q),2)

# 各季節と分位数ごとにピンボールスコアを計算
def pinball_score(modelling_table):
    seasons = ['spring', 'summer', 'autumn', 'winter']
    annual_scores = [] 
    for season in seasons:
        score = [] # 各季節の平均ピンボールスコアを格納
        for quantile in range(10, 100, 10):
            alpha = quantile / 100
            q = modelling_table[f"{season}_q{quantile}"]
            y = modelling_table["total_generation_MWh"]
            pinball_loss = pinball(y, q, alpha).mean()
            rounded_loss = round(pinball_loss, 2) # 小数点2桁に丸める
            score.append(rounded_loss)
        average_loss = round(sum(score) / len(score), 2)
        annual_scores.append(average_loss)
        print(f"{season} season average pinball loss = {average_loss:.2f}\n")

    # 年間平均ピンボールスコアの計算と表示
    annual_average_loss = round(sum(annual_scores) / len(annual_scores), 2)
    print(f"annual_average_loss = {annual_average_loss:.2f}")

# ピンボールスコアの計算と表示
pinball_score(modelling_table)

spring season average pinball loss = 27.50

summer season average pinball loss = 30.45

autumn season average pinball loss = 26.75

winter season average pinball loss = 24.01

annual_average_loss = 27.18
